In [ ]:
import json
import numpy as np
import cv2
import os

# --- Utility Functions ---
def load_json(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

def blend_images(base, overlay):
    gray = cv2.cvtColor(overlay, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
    mask_inv = cv2.bitwise_not(mask)
    bg = cv2.bitwise_and(base, base, mask=mask_inv)
    fg = cv2.bitwise_and(overlay, overlay, mask=mask)
    return cv2.add(bg, fg)

def compute_homography(sift, matcher, img1, img2):
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)
    matches = matcher.knnMatch(des2, des1, k=2)
    good = [m for m, n in matches if m.distance < 0.75 * n.distance]
    if len(good) > 10:
        src_pts = np.float32([kp2[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp1[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)
        H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
        return H
    else:
        raise ValueError("Not enough matches for homography.")

def detect_lanes_and_overlay(image): 
    if image is None:
        return image
    
    height, width = image.shape[:2]
    output = image.copy()

    x_offset = 223
    cropped = image[:, x_offset:-x_offset]
    cropped_width = cropped.shape[1]

    region_height = 250
    cropped_height = cropped.shape[0]
    regions = {
        "Top": (0, region_height)
    }
#     regions = {
#         "Top": (0, region_height),
#         "Bottom": (cropped_height - region_height, cropped_height)
#     }

    for y_start, y_end in regions.values():
        region = cropped[y_start:y_end, :]
        gray = cv2.cvtColor(region, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        edges = cv2.Canny(blurred, 50, 150)

        lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=50, maxLineGap=20)
        if lines is not None:
            for line in lines:
                x1, y1, x2, y2 = line[0]
                x1_full = x1 + x_offset
                x2_full = x2 + x_offset
                y1_full = y1 + y_start
                y2_full = y2 + y_start
                cv2.line(output, (x1_full, y1_full), (x2_full, y2_full), (0, 255, 0), 2)

    return output

# --- Setup ---
dataset_dir = r"C:\Users\ASUS\Labs\IVP\Dataset"
sample_data_file = os.path.join(dataset_dir, "v1.0-mini", "sample_data.json")
sample_data = load_json(sample_data_file)

front_keys = {
    "front": "sweeps/CAM_FRONT/",
    "front_left": "sweeps/CAM_FRONT_LEFT/",
    "front_right": "sweeps/CAM_FRONT_RIGHT/"
}
back_keys = {
    "back": "sweeps/CAM_BACK/",
    "back_left": "sweeps/CAM_BACK_RIGHT/",
    "back_right": "sweeps/CAM_BACK_LEFT"
}

def group_and_sort_frames(keys):
    grouped = {key: [] for key in keys}
    for sample in sample_data:
        for key, path in keys.items():
            if path in sample["filename"]:
                grouped[key].append(sample)
    for key in grouped:
        grouped[key] = sorted(grouped[key], key=lambda x: x["timestamp"])
    return grouped

front_frames = group_and_sort_frames(front_keys)
back_frames = group_and_sort_frames(back_keys)

ref_paths = {
    "back": os.path.join(dataset_dir, r"sweeps\CAM_BACK\n008-2018-08-01-15-16-36-0400__CAM_BACK__1533151609637558.jpg"),
    "back_left": os.path.join(dataset_dir, r"sweeps\CAM_BACK_RIGHT\n008-2018-08-01-15-16-36-0400__CAM_BACK_RIGHT__1533151609378113.jpg"),
    "back_right": os.path.join(dataset_dir, r"sweeps\CAM_BACK_LEFT\n008-2018-08-01-15-16-36-0400__CAM_BACK_LEFT__1533151609447405.jpg")
}
ref_paths.update({
    "front": os.path.join(dataset_dir, front_frames["front"][0]["filename"].replace("/", "\\")),
    "front_left": os.path.join(dataset_dir, front_frames["front_left"][0]["filename"].replace("/", "\\")),
    "front_right": os.path.join(dataset_dir, front_frames["front_right"][0]["filename"].replace("/", "\\"))
})

ref_images = {k: cv2.imread(p) for k, p in ref_paths.items()}
if any(img is None for img in ref_images.values()):
    raise ValueError("Failed to load one or more reference images.")

sift = cv2.SIFT_create()
matcher = cv2.FlannBasedMatcher(dict(algorithm=1, trees=5), dict(checks=50))

H_front_left = compute_homography(sift, matcher, ref_images["front"], ref_images["front_left"])
H_front_right = compute_homography(sift, matcher, ref_images["front"], ref_images["front_right"])
H_back_left = compute_homography(sift, matcher, ref_images["back"], ref_images["back_left"])
H_back_right = compute_homography(sift, matcher, ref_images["back"], ref_images["back_right"])

h, w = ref_images["front"].shape[:2]
canvas_h, canvas_w = h * 2, w * 5
offset_x, offset_y = w * 2, h // 2
T = np.array([[1, 0, offset_x], [0, 1, offset_y], [0, 0, 1]], dtype=np.float32)

H_front_left_warp = (T @ H_front_left).astype(np.float32)
H_front_right_warp = (T @ H_front_right).astype(np.float32)
H_back_left_warp = (T @ H_back_left).astype(np.float32)
H_back_right_warp = (T @ H_back_right).astype(np.float32)

source_points =  np.float32([
    [3000, 1200], [5000, 1200],
    [1500, 1600], [6500, 1600]
])
destination_points = np.float32([
    [300, 0], [700, 0],
    [300, 800], [700, 800]
])
bev_H = cv2.getPerspectiveTransform(source_points, destination_points)

ego_car_path = r"C:\Users\ASUS\Downloads\download-removebg-preview.png"
ego_car = cv2.imread(ego_car_path, cv2.IMREAD_UNCHANGED)
if ego_car is None:
    raise ValueError("Ego car image not found!")

cv2.namedWindow("Combined BEV", cv2.WINDOW_AUTOSIZE)
num_frames = min(len(front_frames["front"]), len(front_frames["front_left"]), len(front_frames["front_right"]),
                 len(back_frames["back"]), len(back_frames["back_left"]), len(back_frames["back_right"]))

for i in range(num_frames):
    def load_cam_images(keys, frames):
        return {
            key: cv2.imread(os.path.join(dataset_dir, frames[key][i]["filename"].replace("/", "\\")))
            for key in keys
        }

    front_imgs = load_cam_images(front_keys, front_frames)
    back_imgs = load_cam_images(back_keys, back_frames)

    if any(img is None for img in list(front_imgs.values()) + list(back_imgs.values())):
        continue

    f_base = cv2.warpPerspective(front_imgs["front"], T, (canvas_w, canvas_h))
    f_left = cv2.warpPerspective(front_imgs["front_left"], H_front_left_warp, (canvas_w, canvas_h))
    f_right = cv2.warpPerspective(front_imgs["front_right"], H_front_right_warp, (canvas_w, canvas_h))
    f_canvas = blend_images(f_base, f_left)
    f_canvas = blend_images(f_canvas, f_right)
    f_bev = cv2.warpPerspective(f_canvas, bev_H, (1000, 800))

    b_base = cv2.warpPerspective(back_imgs["back"], T, (canvas_w, canvas_h))
    b_left = cv2.warpPerspective(back_imgs["back_left"], H_back_left_warp, (canvas_w, canvas_h))
    b_right = cv2.warpPerspective(back_imgs["back_right"], H_back_right_warp, (canvas_w, canvas_h))
    b_canvas = blend_images(b_base, b_left)
    b_canvas = blend_images(b_canvas, b_right)
    b_bev = cv2.warpPerspective(b_canvas, bev_H, (1000, 800))

    b_bev_flipped = cv2.flip(cv2.flip(b_bev, 0), 1)
    combined_bev = np.vstack([f_bev, b_bev_flipped])

    car_target_height = 900
    car_target_width = int(car_target_height * ego_car.shape[1] / ego_car.shape[0]) - 250
    ego_car_resized = cv2.resize(ego_car, (car_target_width, car_target_height))

    x_offset = (combined_bev.shape[1] - car_target_width) // 2
    y_offset = (combined_bev.shape[0] - car_target_height) // 2

    alpha = ego_car_resized[:, :, 3] / 255.0
    for c in range(3):
        combined_bev[y_offset:y_offset+car_target_height, x_offset:x_offset+car_target_width, c] = (
            (1 - alpha) * combined_bev[y_offset:y_offset+car_target_height, x_offset:x_offset+car_target_width, c] +
            alpha * ego_car_resized[:, :, c]
        ).astype(np.uint8)

    combined_bev = cv2.resize(combined_bev, (1000, 1000))
    combined_bev = combined_bev[:, 190:-190]

    # --- Detect and Draw Lanes ---
    combined_bev = detect_lanes_and_overlay(combined_bev)

    cv2.imshow("Combined BEV", combined_bev)
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
